In [78]:
import random
import math
import numpy as np

In [22]:
def create_cdf(pdf):
    acc = 0
    cdf = []
    for p in pdf:
        acc += p
        cdf.append(acc)
    return cdf

def generate_from_cdf(cdf):
    u = random.random()
    for i, c in enumerate(cdf):
        if u < c:
            return i
    return -1

In [65]:
class PhaseType:
    def __init__(self, pi, d0):
        self.n_rates = len(pi)
        self.pi = pi
        self.d0 = d0
        self.state_cdf = create_cdf(self.pi)
        self.t_matrix = [[(x if x > 0 else 0) for x in row] for row in self.d0]
        int_trates = [sum([(-x if x < 0 else 0) for x in row]) for row in self.d0]
        ex_trates = [sum(row) for row in self.t_matrix]
        # absorption rates
        self.a_rates = [x-y for x,y in zip(int_trates, ex_trates)]
        
    def generate(self,n=1):
        if n > 1:
            out_list = []
            for i in range(n):
                out_list += self.generate()
            return out_list
        cur_state = generate_from_cdf(self.state_cdf)
        t = 0
        while cur_state != self.n_rates:
            transitions = self.t_matrix[cur_state]
            abs_rate = self.a_rates[cur_state]
            total_rate = sum(transitions) + abs_rate
            t_pdf = [t/total_rate for t in transitions] + [abs_rate/total_rate]
            t_cdf = create_cdf(t_pdf)
            t += generate_exp(1, total_rate)[0]
            cur_state = generate_from_cdf(t_cdf)
        return [t]

In [66]:
def generate_exp(n, rate):
    out_list = []
    for i in range(n):
        u = random.random()
        out_list.append(-math.log(u)/rate)
    return out_list

In [118]:
erlang_pi = [1.0, 0.0]
erlang_d = [[-1.0, 1.0],
            [0, -1.0]]
erlang_d = PhaseType(erlang_pi, erlang_d)

In [119]:
erlangs = erlang_d.generate(5000)

In [120]:
sum(erlangs)/5000

2.045819193607778

In [121]:
sum(erlangs)

10229.095968038891

In [122]:
np.array(erlangs).std()

1.4497401280077278

In [123]:
math.sqrt(2)

1.4142135623730951